# Modifications after Webex Meeting with Dow Team + Dr. Irfan Khan, May 19, 2020: Tolman Correction to Interfacial Tension

On Tuesday, May 19, 2020, I met with members of our Dow team and Dow scientist Dr. Irfan Khan over Webex to discuss next steps for improving this model of bubble growth. This notebook implements and records the results of one those improvements: the Tolman correction to the Young-Laplace formula for the interfacial tension.

Dr. Valeriy Ginzburg noted that the interfacial tension can be reduced by up to a factor of 2 for 5 nm bubbles in this case. This can be captured by the Tolman model of interfacial tension, which includes a first-order correction to the Young-Laplace equation due to the effect of a small radius of curvature,

\begin{align*}
    \frac{\gamma(r)}{\gamma(\infty)} &= \frac{1}{1+ \frac{2\delta}{r}}\\
\end{align*}

where $\gamma(r)$ is the interfacial tension along a surface with a radius of curvature of $r$, and $\delta$ is the Tolman length, which can be calculated *a priori* but is typically estimated empirically. This can be substituted into the Young-Laplace equation, $p_{bubble} = p + \frac{2 \gamma(R)}{R}$, although there are additional corrections that become important when $R \sim \delta$ (see Tolman *JCP* 1949).

**NOTE: this file also uses 1k3f_22c.csv for the polyol data, rather than 1k3f_30c.csv, as was used in some other IPython notebooks. This file thus uses interfacial tension estimates made at the appropriate temperature (22 C instead of 30 C).**

In [7]:
# adds custom libraries to path
import sys
sys.path.append('../libs/')
sys.path.append('src/')

# import libraries
import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.interpolate

# custom libraries
import bubble
import flow
import plot
import analytics as an

from importlib import reload
reload(bubble)
reload(an)

# lists data files required
# load data (interfacial tension, solubility, and diffusivity) for polyols? we don't have data for VORANOL 360...use 1k3f
polyol_data_file = 'input/1k3f_22c.csv' # from DFT and PC-SAFT estimates
# load CO2 equation of state data for desired temperature [csv]
eos_co2_file = 'input/eos_co2_22-0C.csv'
# folder to save figures
save_folder = 'figs/'
save_figs = True

# system parameters
eta_o = 3.6 # viscosity of pure-polyol in outer stream, VORANOL 360, 'Polyol Properties.ppt'  [Pa.s]
p_in = 85E5 # estimate of pressure at capillary in let [Pa]
L = 0.099 # length of observation capillary, p. 154 of lab notebook #0005 [m]
p_atm = 101.3E3 # atmospheric pressure, assumed to be outlet pressure [Pa]
Q_o = 2000/60E9 # outer stream flow rate, input to ISCO 260 D [m^3/s]
Q_i = 180/60E9 # inner stream flow rate, input to ISCO 100 DM [m^3/s]
R_cap = 254E-6 # radius of inner walls of observation capillary, PFA 0.002" ID [m]
R_nuc = 4E-9 # initial bubble radius, estimated as a bit larger than prediction in slide 13 of 20200305_aps_march_meeting [m]
p_s = 72E5 # saturation pressure of CO2 in VORANOL 360, p. 154 of lab notebook #0005 [Pa]

# numerical parameters
dt = 1E-7 # time step for numerical model [s]

# CONVERSIONS
s_2_ms = 1000
s_2_us = 1E6
kg_2_g = 1E3
kg_2_ng = 1E12
kgm3_2_gmL = 1E-3
m_2_mm = 1000
m_2_um = 1E6
m_2_nm = 1E9
Pa_2_bar = 1E-5
Pa_2_MPa = 1E-6
Nm_2_mNm = 1000

### Solve for Flow Profile

Solve Stokes flow and constraints from conservation of volume for inner and outer streams.

In [8]:
# calculates inner stream velocity based on flow rates, viscosities, and observation capillary inner diameter
eta_i0 = 0.8
R_i0 = 50E-6
v0 = 0.5
x0 = [eta_i0, R_i0, v0] # initial guesses for p_in [Pa], v [m/s], R_i [m]
ordering = [0, 3, 4, 5, 6, 7, 8, 1, 9, 2] # indices of vars in alphabetical order from list vars + args
args = (eta_o, L, p_atm, p_in, Q_i, Q_o, R_cap, ordering)
soln = scipy.optimize.root(flow.sheath_eqns_input, x0, args=(args,))
eta_i, R_i, v = soln.x

print('Inner stream viscosity eta_i = {0:.2f} Pa.s'.format(eta_i))
print('Inner stream velocity v = {0:.1f} mm/s'.format(v*m_2_mm))
print('Radius of inner stream R_i = {0:.1f} um.'.format(R_i*m_2_um))

Inner stream viscosity eta_i = 0.91 Pa.s
Inner stream velocity v = 422.5 mm/s
Radius of inner stream R_i = 49.2 um.


### Plot Example

We first plot an example since we are using a different set of values for the interfacial tension than in previous notebooks.

In [9]:
reload(bubble)
t_nuc = 0.068
dt = 1E-6
# computes bubble growth trajectory using modified Epstein-Plesset model
t, m, D, p, p_bubble, if_tension, c_s, R, rho_co2 = bubble.grow(dt, t_nuc, p_s, R_nuc, L, p_in, v,
                                                                            polyol_data_file, eos_co2_file, 
                                                                            adaptive_dt=True, implicit=False)

FileNotFoundError: [Errno 2] File input/1k3f_30c_sensitivity/1k3f_30c.csv does not exist: 'input/1k3f_30c_sensitivity/1k3f_30c.csv'

Below, we plot the results to compare to previous notebooks.

In [ ]:
props_list = (R, m, p, p_bubble, rho_co2, if_tension)
ax = plot.all_props(t, t_nuc, props_list, x_log=False, title='New Interfacial Tension Data')

### Add Tolman Correction

We now look at what happens to the properties when we add a Tolman length correction of 10 nm.

In [ ]:
reload(bubble)
t_nuc = 0.068
dt = 1E-6
# computes bubble growth trajectory using modified Epstein-Plesset model
t, m, D, p, p_bubble, if_tension, c_s, R, rho_co2 = bubble.grow(dt, t_nuc, p_s, R_nuc, p_atm, L, p_in, v,
                                                                            polyol_data_file, eos_co2_file, 
                                                                            adaptive_dt=True, implicit=False, d_tolman=10E-9)

Plot results with Tolman correction.

In [ ]:
props_list = (R, m, p, p_bubble, rho_co2, if_tension)
ax = plot.all_props(t, t_nuc, props_list, x_log=True, title='Tolman Length = 10 nm')

### Vary the Tolman Length

I will look at Tolman lengths of 0 nm (no Tolman correction), 5 nm, 10 nm, and 20 nm.

In [ ]:
reload(an)
reload(bubble)

# explicit time-stepping for different Tolman lengths [m], with the reference value (0 nm) first for later analysis
d_tolman_list = [0, 5E-9, 10E-9, 20E-9, 100E-9]
dt0 = 1E-8
t_nuc = 0.068
adaptive_dt = True
implicit = False
tol_R = 1E-3
alpha = 0.8

args = (dt0, t_nuc, p_s, R_nuc, p_atm, L, p_in, v, polyol_data_file, eos_co2_file, adaptive_dt, implicit, tol_R, alpha)
tolman_sweep = an.sweep(d_tolman_list, an.d_tolman, args, param_name='d_tolman', conv=m_2_nm, units='nm')

We extract parameters to plot.

In [ ]:
# extracts results
t_list = tolman_sweep[0]
R_list = tolman_sweep[-2]
p_list = tolman_sweep[3]
p_bubble_list = tolman_sweep[4]
if_tension_list = tolman_sweep[5]
rho_co2_list = tolman_sweep[-1]

Next, we compare the results in a plot.

In [ ]:
label_list = [r'$d_{Tolman}$' + ' = {0:.1f} nm'.format(d_tolman*m_2_nm) for d_tolman in d_tolman_list]
ax = plot.series(t_list, R_list, label_list, t_nuc, s_2_ms, m_2_um, r'$t - t_{nuc}$ [ms]',
           r'$R$ [$\mu$m]', 'Varying Tolman Length, Explicit', x_log=True, y_log=True)

Next, we plot the difference in the result from the reference with a Tolman length of 0 (no correction).

In [ ]:
reload(plot)
diff_label_list = label_list
diff_label_list[0] = r'ref, $d_{Tolman}$' + ' = {0:.1f} nm'.format(d_tolman_list[0]*m_2_nm)
ax = plot.diff(t_list, R_list, diff_label_list, t_nuc, s_2_ms, m_2_um, r'$t - t_{nuc}$ [ms]',
           r'$R$ [$\mu$m]', 'Effect of Tolman Length, Explicit', x_log=True, y_log=True)

We'll also plot the fractional difference from the reference.

In [ ]:
reload(plot)
ax = plot.diff(t_list, R_list, diff_label_list[1:], t_nuc, s_2_ms, 1, r'$t - t_{nuc}$ [ms]',
           r'$\Delta R/R_{ref}$ [nondim]', 'Tolman Correction, Explicit', x_log=True, y_log=True, frac=True)
ax.set_ylim([1E-5, 1])

The effect of the Tolman length looks no bigger than the tolerance on $R$ (0.001). Is the Tolman length being implemented?

To check, I will plot the interfacial tension.

### Interfacial Tension

In [ ]:
label_list = [r'$d_{Tolman}$' + ' = {0:.1f} nm'.format(d_tolman*m_2_nm) for d_tolman in d_tolman_list]
ax = plot.series(t_list, if_tension_list, label_list, t_nuc, s_2_ms, Nm_2_mNm, r'$t - t_{nuc}$ [ms]',
           r'$\gamma$ [mN/m]', 'Varying Tolman Length, Explicit', x_log=True, y_log=True)

The Tolman length correction is definitely affecting the interfacial tension as expected, but perhaps the interfacial tension is already too small to have a profound effect on the bubble growth. I will plot as a function of the radius to double-check that the convergence of the Tolman-corrected interfacial tension and the Young-Laplace formula for the interfacial tension ($d_{Tolman} = 0$ nm) occurs around the Tolman length.

In [ ]:
ax = plot.series(R_list, if_tension_list, label_list, 0, m_2_nm, Nm_2_mNm, r'$R$ [nm]',
           r'$\gamma$ [mN/m]', 'Varying Tolman Length, Explicit', x_log=True, y_log=True)

The convergence appears to occur when the radius is at least 200 times larger than the Tolman length. I'm curious, so I will also plot the fractional difference.

In [ ]:
reload(plot)
ax = plot.diff(R_list, if_tension_list, diff_label_list[1:], 0, m_2_nm, 1, r'$R$ [nm]',
               r'$\Delta \gamma/\gamma_{ref}$ [nondim]', 'Fractional Tolman Correction, Explicit', 
               x_log=True, y_log=True, frac=True)
ax.plot([1, 1E7], [0.01, 0.01], 'k--')

Here, the black line marks the value of 1%. The Tolman correction should be less than 1% for $R > 200d_{Tolman}$ (= $2d_{Tolman}/R$). This appears to be the case, indicating that the Tolman correction is indeed implemented correctly.

### Why is there a plateau at the beginning?

We see a plateau in the interfacial tension at the beginning of the plot that is two plots above this cell. Why is that? My hypothesis is that the planar interfacial tension does not change substantially for the range of densities in that range of radius. To test this hypothesis, I will plot the interfacial tension and density together.

In [ ]:
i = 1 # Tolman length of 5 nm

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.array(R_list[1])*m_2_um, np.array(if_tension_list[1])*Nm_2_mNm, 'b--', label=r'$\gamma(R)$ [mN/m]')
ax.plot(np.array(R_list[1])*m_2_um, np.array(rho_co2_list[1])*kgm3_2_gmL, 'g-', label=r'$\rho_{CO2}(R)$ [g/mL]')
ax.set_xlabel(r'$R$ [$\mu$m]', fontsize=16)
ax.set_title('Plateau in Tolman-corrected Interfacial Tension in the Context of CO2 Density')
ax.set_xscale('log')
ax.set_yscale('log')

plt.legend()



We see that the plateau occurs after the radius $R$ far exceeds the Tolman length $d_{Tolman}$ (5 nm in the above plot), but the density of CO2 is still liquid and not changing. In the above plot, this plateau regime occurs from a radius of about 100 nm to 1 $\mu$m.

In [ ]:
i = 1 # Tolman length of 5 nm

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.array(t_list[1])*s_2_ms, np.array(if_tension_list[1])*Nm_2_mNm, 'b--', label=r'$\gamma(R)$ [mN/m]')
ax.plot(np.array(t_list[1])*s_2_ms, np.array(rho_co2_list[1])*kgm3_2_gmL, 'g-', label=r'$\rho_{CO2}(R)$ [g/mL]')
ax.set_xlabel(r'$R$ [$\mu$m]', fontsize=16)
ax.set_title('Plateau in Tolman-corrected Interfacial Tension in the Context of CO2 Density')
ax.set_xscale('log')
ax.set_yscale('log')

plt.legend()